<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/attention-and-transformers-mechanism/gpt-mechanism/Building_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

##Setup

In [ ]:
# Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F

##Dataset

In [3]:
# read it in to inspect it
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for i, ch in enumerate(chars)}

# encoder: take a string, output a list of integers
encode = lambda s: [char_to_idx[c] for c in s]
# decoder: take a list of integers, output a string
decode = lambda l: "".join([idx_to_char[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
# Let's now split up the data into train and validation sets
n = int(0.9 * len(data))  # first 90% will be train, rest val

train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1: block_size + 1]

for t in range(block_size):
  context = x[: t + 1]
  target = y[t]
  print(f"when input is {context} then target: {target}")

when input is tensor([18]) then target: 47
when input is tensor([18, 47]) then target: 56
when input is tensor([18, 47, 56]) then target: 57
when input is tensor([18, 47, 56, 57]) then target: 58
when input is tensor([18, 47, 56, 57, 58]) then target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target: 58


In [15]:
torch.manual_seed(1337)

batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  x = torch.stack([data[i: i + block_size] for i in ix])
  y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
  return x, y

In [17]:
x_batch, y_batch = get_batch("train")
print(f"inputs:\n\t{x_batch.shape} \n\t{x_batch}")
print(f"targets:\n\t{y_batch.shape} \n\t{y_batch}")
print('-------------------------------------')

for b in range(batch_size):  # batch dimension
  for t in range(block_size): # time dimension
    context = x_batch[b, :t + 1]
    target = y_batch[b, t]
    print(f"when input is {context.tolist()} then target: {target}")

inputs:
	torch.Size([4, 8]) 
	tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])
targets:
	torch.Size([4, 8]) 
	tensor([[43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])
-------------------------------------
when input is [57] then target: 43
when input is [57, 43] then target: 60
when input is [57, 43, 60] then target: 43
when input is [57, 43, 60, 43] then target: 52
when input is [57, 43, 60, 43, 52] then target: 1
when input is [57, 43, 60, 43, 52, 1] then target: 63
when input is [57, 43, 60, 43, 52, 1, 63] then target: 43
when input is [57, 43, 60, 43, 52, 1, 63, 43] then target: 39
when input is [60] then target: 43
when input is [60, 43] then target: 42
when input is [60, 43, 42] then target: 8
when input is [60, 43, 42, 8] then target: 0
when input is 

In [18]:
# our input to the transformer
print(x_batch)

tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])


##Language Model